## Springboard Capstone 1.0
## SP500 Emini Swing Analysis

This study is to analyze the price swings of the SP500 e-mini contract. Price swings move 
Final Project Proposal: “SP500 e-Mini Daytrading - Optimizing Pullback Entries” 

The SP500 e-Mini contract volatility has increased since Donald Trump became the U.S. President. The length of swings (measured in ticks) has increased and the number of swings per day has also increased. The question for the typical day trader is how/where do I enter a trade and what should my profit target be? A day trader needs some basic e-Mini swing descriptive statistics to plan and execute his day trading approach.

This proposed study will do 2 things:

1)	Develop descriptive statistics for SP500 e-mini 1 minute bar swings. (Note a swing high will be defined as high bar with 2 lower or equal bars before and after the swing high point and a similar approach for swing lows).  For example – how many ticks is a typical swing, how long does a swing go (in bars / time) before a pull back, does the swing have a breakout, if the swing has a breakout, how many bars does it go and how far past the previous swing high / low ? 

As shown in Figure 1 below, when the e-mini market seems to be moving in an “organized pattern, the market makes a series of higher highs and higher lows until the market reverses and makes a series of lower lows and lower highs. As a day trader, knowing that the down swings from 11:00 am to 12:00 pm EST typically average 20+ ticks in length down, 13 – 15 tick pull backs, and the breakouts below the previous swing lows only last for 1 – 2 bars is useful information. Figure 2 shows a similar structure.

Figure 3 is also a down trend however it’s more sideways trading. Seeing the swing lengths and number of bars in each swing (both with and against the trend) lets the trader develop an understanding of the current market environment and establish reasonable entry and stops.

2)	Optimize a pullback entry approach. Figures 1 – 3 all contain 3 exponential moving averages (ema). The blue line (21 ema) is a typical entry point and the red line (34 ema) and magenta (55 ema) show an approximate swing retracement area before the market continues in the larger trend direction.

My hypothesis is that a pull back entry at the 21 ema with a stop X ticks beyond the 55 ema can be profitable in the SP500 emini. I will need to leverage the existing backtesting code provided in our certificate program to develop pullback (i.e. limit entries) and then optimize these entries based upon the descriptive statistics analyses developed in step 1 above.

I will use the SP e-mini contract from Sept 2018 – Dec 2018 for my testing. 

Fig 1:
![SP500 1 min chart] (https://github.com/tom1presto/Algo-Trading-Certificate-Program/blob/master/SP500%20Study%20-%20Fig%201.png)

Fig 2:
![SP500 1 min chart] (https://github.com/tom1presto/Algo-Trading-Certificate-Program/blob/master/SP500%20Study%20-%20Fig%202.png)

Fig 3:
![SP500 1 min chart] (https://github.com/tom1presto/Algo-Trading-Certificate-Program/blob/master/SP500%20Study%20-%20Fig%203.png)

In [1]:
# Import Libraries
#
import pandas as pd
import numpy as np
import plotly.offline as pyo
import plotly.graph_objs as go
from plotly import __version__
print(__version__)

from datetime import datetime

# current date and time
now = datetime.now()
print(now)

3.7.0
2019-05-14 16:54:11.051284


In [3]:
# Reading ES 03 19 - Jan 1 - 18
# 
df = pd.read_csv(r"C:\Users\xxxxxx\Desktop\ES_03_19_2019_01_to_18_20secV2.csv", 
                    index_col=0, parse_dates=True, sep=",")
#  names=['date', 'open', 'high','low','open','volume'],|
df.head(10)

,open,high,low,close
Date_Time,,,,
2019-01-01 23:00:00,2493.00,2493.00,2492.75,2493.00
2019-01-01 23:00:20,2493.25,2493.50,2493.25,2493.50
2019-01-01 23:00:40,2493.25,2493.25,2493.00,2493.25
2019-01-01 23:01:00,2493.50,2493.50,2493.25,2493.25
2019-01-01 23:01:20,2493.25,2493.50,2493.25,2493.25
2019-01-01 23:01:40,2493.25,2493.25,2492.75,2492.75
2019-01-01 23:02:00,2492.75,2492.75,2492.75,2492.75
2019-01-01 23:02:20,2492.50,2492.50,2492.00,2492.00
2019-01-01 23:02:40,2492.00,2492.00,2491.75,2491.75


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 72361 entries, 2019-01-01 23:00:00 to 2019-01-18 17:00:00
Data columns (total 4 columns):
open     51953 non-null float64
high     51953 non-null float64
low      51953 non-null float64
close    51953 non-null float64
dtypes: float64(4)
memory usage: 2.8 MB


In [5]:
df.tail(10)

,open,high,low,close
Date_Time,,,,
2019-01-18 16:57:00,2676.25,2676.50,2676.25,2676.50
2019-01-18 16:57:20,2676.25,2676.50,2676.00,2676.25
2019-01-18 16:57:40,2676.00,2676.75,2676.00,2676.50
2019-01-18 16:58:00,2676.50,2676.75,2676.50,2676.75
2019-01-18 16:58:20,2676.50,2677.25,2676.50,2676.75
2019-01-18 16:58:40,2676.75,2677.25,2676.25,2676.75
2019-01-18 16:59:00,2676.75,2677.50,2676.75,2677.25
2019-01-18 16:59:20,2677.00,2677.00,2676.25,2676.25
2019-01-18 16:59:40,2676.25,2677.25,2676.25,2677.00


The 21 and 55 exponential moving average are key to this study. The 55 EMA will set the trend and pull backs to the 21 EMA will be used for entries

In [6]:
df['21ema'] = pd.Series.ewm(df['close'], span=21).mean()
df['55ema'] = pd.Series.ewm(df['close'], span=55).mean()

In [7]:
df.tail(10)

,open,high,low,close,21ema,55ema
Date_Time,,,,,,
2019-01-18 16:57:00,2676.25,2676.50,2676.25,2676.50,2675.190774,2674.365863
2019-01-18 16:57:20,2676.25,2676.50,2676.00,2676.25,2675.287105,2674.436017
2019-01-18 16:57:40,2676.00,2676.75,2676.00,2676.50,2675.397408,2674.512751
2019-01-18 16:58:00,2676.50,2676.75,2676.50,2676.75,2675.520412,2674.595805
2019-01-18 16:58:20,2676.50,2677.25,2676.50,2676.75,2675.632226,2674.675664
2019-01-18 16:58:40,2676.75,2677.25,2676.25,2676.75,2675.733869,2674.752457
2019-01-18 16:59:00,2676.75,2677.50,2676.75,2677.25,2675.871733,2674.844798
2019-01-18 16:59:20,2677.00,2677.00,2676.25,2676.25,2675.906129,2674.896686
2019-01-18 16:59:40,2676.25,2677.25,2676.25,2677.00,2676.005592,2674.974259


## df_mini copy for testing sample

In [8]:
#
# Copying subset of data for testing
#
df_mini = df.iloc[-3000:].copy()

In [9]:
df_mini.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 3000 entries, 2019-01-18 00:20:20 to 2019-01-18 17:00:00
Data columns (total 6 columns):
open     2920 non-null float64
high     2920 non-null float64
low      2920 non-null float64
close    2920 non-null float64
21ema    3000 non-null float64
55ema    3000 non-null float64
dtypes: float64(6)
memory usage: 164.1 KB


In [10]:
df_mini.tail(10)

,open,high,low,close,21ema,55ema
Date_Time,,,,,,
2019-01-18 16:57:00,2676.25,2676.50,2676.25,2676.50,2675.190774,2674.365863
2019-01-18 16:57:20,2676.25,2676.50,2676.00,2676.25,2675.287105,2674.436017
2019-01-18 16:57:40,2676.00,2676.75,2676.00,2676.50,2675.397408,2674.512751
2019-01-18 16:58:00,2676.50,2676.75,2676.50,2676.75,2675.520412,2674.595805
2019-01-18 16:58:20,2676.50,2677.25,2676.50,2676.75,2675.632226,2674.675664
2019-01-18 16:58:40,2676.75,2677.25,2676.25,2676.75,2675.733869,2674.752457
2019-01-18 16:59:00,2676.75,2677.50,2676.75,2677.25,2675.871733,2674.844798
2019-01-18 16:59:20,2677.00,2677.00,2676.25,2676.25,2675.906129,2674.896686
2019-01-18 16:59:40,2676.25,2677.25,2676.25,2677.00,2676.005592,2674.974259


In [11]:
df_mini.head()

,open,high,low,close,21ema,55ema
Date_Time,,,,,,
2019-01-18 00:20:20,2641.50,2641.50,2641.50,2641.50,2641.301647,2641.265153
2019-01-18 00:20:40,2641.25,2641.25,2641.00,2641.25,2641.296438,2641.264559
2019-01-18 00:21:00,NaN,NaN,NaN,NaN,2641.296438,2641.264559
2019-01-18 00:21:20,2641.50,2641.75,2641.50,2641.75,2641.345767,2641.284195
2019-01-18 00:21:40,2641.75,2641.75,2641.75,2641.75,2641.388959,2641.302948


This study will set the trend based upon the current bar being "x points" above or below the 55 EMA. A trend is long when the high is the current
bar is "trend points (set at 1.5)" above the 55EMA. The trend is stays long until the low of a bar is "trend points" below the 55 EMA. The trend
then stays short until the long trend condition is set.

In [13]:
# Code works 2019-03-19 - moving above and dropping print statements
# 
# testing writing multiple dict in a list
#
# DF [0]open, [1]high, [2]low, [3]close, [4]21ema, [5]55ema   
#
trend_change_up_plot = {}    # dict with the trend change up price to plot 
trend_change_dn_plot = {}    # dict with the trend change down price to plot
trend_change_flag = {}       # either 1 (long) or -1 (short)
prev_trend = 1
current_trend = 1
trend_pts = 1.5              # trend points determines the trend

for i, row in df_mini.iterrows():
    current_bar = i
    if df_mini.index.get_loc(i) == 0:          # initial run through - set entries for first bar
        #print(f'Initial Row: {df_test.index.get_loc(i)} - Current Bar {current_bar} 21EMA: {row[4]:.2f} 55EMA {row[5]:.2f}')
        trend_change_up_plot[i] = row[3]   # set initial dict to have close value
        trend_change_flag[i] = 1     

        ##--------- Down Trend Change --------------------------------------------------------------------------------#    
    if current_trend == 1:                 # assume initial upswing - check 
        if row[2] <= row[5] - trend_pts:   # if low < 55EMA - trend points
            #print(f'  Down Trend: {df_test.index.get_loc(i)} {current_bar}: Low {row[2]:.2f} <= {row[5] - trend_pts:.2f} === 55EMA {row[5]:.2f} - {trend_pts}')
            prev_trend = current_trend
            trend_change_dn_plot[i] = row[2]   # set initial dict to have low
            trend_change_flag[i] =  -1   
            current_trend = -1
    
    ##--------- Up Trend Change --------------------------------------------------------------------------------#    
    if current_trend == -1:                # 
        if row[1] >= row[5] + trend_pts:   # if high > 55EMA + trend points
            #print(f'Up Trend: {df_test.index.get_loc(i)} {current_bar}: High {row[1]:.2f} >= {row[5] + trend_pts:.2f} === 55EMA {row[5]:.2f} + {trend_pts}')
            #print('')
            prev_trend = current_trend
            trend_change_up_plot[i] = row[1]
            trend_change_flag[i] =  1  
            current_trend = 1
            
#print(trend_change_up_plot)
#print(trend_change_dn_plot)
# print(trend_change_flag)

# print(trend_change_flag, file=open('testflag.txt', 'w'))

In [14]:
#
# Convert Trend Change up/down and trend change flags to dataframes. These data frames will be used to plot the trend changes on the price chart
#
df_trend_change_up_plot = pd.DataFrame.from_dict(trend_change_up_plot,orient='index',columns=['Price'])
df_trend_change_up_plot.index.names= ['Date_Time']  
#print(df_trend_change_up_plot)
#print(df_trend_change_up_plot.index)

df_trend_change_dn_plot = pd.DataFrame.from_dict(trend_change_dn_plot,orient='index',columns=['Price'])
df_trend_change_dn_plot.index.names= ['Date_Time']  
#print(df_trend_change_dn_plot)
#print(df_trend_change_dn_plot.index)

df_trend_change_flag = pd.DataFrame.from_dict(trend_change_flag,orient='index',columns=['trend'])
df_trend_change_flag.index.names= ['Date_Time']  
#print(df_trend_change_flag)
#print(df_trend_change_flag.index)

In [44]:
# insert merged code here
# print(df_trend_change_flag)

The trend flag dataframe is merged with the price chart data to establish price swings occuring with and against the trend

In [15]:
# 
# Merge trend change dataframe with the price data and reset the index
#
df_mini = pd.merge_ordered(df_mini, df_trend_change_flag, on='Date_Time',how='left',fill_method='ffill')
df_mini.set_index('Date_Time',inplace=True)

#print(df_mini.index)
#print(df_mini.head(20))

A new "up" price swing occurs when the current bar high exceeded the highest hi of the past 5 previous bars. A new "down" swing occurs
when the low of the current bar exceeds the low of the past 5 lowest bars.

The variables below hold the rolling 5 bar max highs and 5 bar min lows and the current bar crosses these values 

In [16]:
#
# set 5 previous bars max hi and min low and when the current bar crosses them
#

df_mini['trail_hi'] = df_mini['high'].rolling(5,min_periods=1).max()
df_mini['trail_hix'] = df_mini['high'] - df_mini['trail_hi'].shift(1)
df_mini['trail_lo'] = df_mini['low'].rolling(5,min_periods=1).min()
df_mini['trail_lox'] = df_mini['trail_lo'].shift(1) - df_mini['low']      
df_mini.tail(10)
#for i in range(200,400):
#    print(df_mini.iloc[i])

,open,high,low,close,21ema,55ema,trend,trail_hi,trail_hix,trail_lo,trail_lox
Date_Time,,,,,,,,,,,
2019-01-18 16:57:00,2676.25,2676.50,2676.25,2676.50,2675.190774,2674.365863,1,2677.00,-0.50,2675.50,-1.25
2019-01-18 16:57:20,2676.25,2676.50,2676.00,2676.25,2675.287105,2674.436017,1,2677.00,-0.50,2675.50,-0.50
2019-01-18 16:57:40,2676.00,2676.75,2676.00,2676.50,2675.397408,2674.512751,1,2676.75,-0.25,2676.00,-0.50
2019-01-18 16:58:00,2676.50,2676.75,2676.50,2676.75,2675.520412,2674.595805,1,2676.75,0.00,2676.00,-0.50
2019-01-18 16:58:20,2676.50,2677.25,2676.50,2676.75,2675.632226,2674.675664,1,2677.25,0.50,2676.00,-0.50
2019-01-18 16:58:40,2676.75,2677.25,2676.25,2676.75,2675.733869,2674.752457,1,2677.25,0.00,2676.00,-0.25
2019-01-18 16:59:00,2676.75,2677.50,2676.75,2677.25,2675.871733,2674.844798,1,2677.50,0.25,2676.00,-0.75
2019-01-18 16:59:20,2677.00,2677.00,2676.25,2676.25,2675.906129,2674.896686,1,2677.50,-0.50,2676.25,-0.25
2019-01-18 16:59:40,2676.25,2677.25,2676.25,2677.00,2676.005592,2674.974259,1,2677.50,-0.25,2676.25,0.00


The code below is the main logic for calculating swings and their length in price bars and points

The logic creates a separate file for up swings and down swings to be plotted on the chart separately

In [17]:
# Using iterrows to calc swings  -   latest version 2019-03-23
#
# 
# index 
# Date/Tim  [0]open, [1]high, [2]low, [3]close, [4]21ema, [5]55ema, [6]trend, [7]trail_hi, [8]trail_hix, [9]trail_lo, [10]trail_lox 
#
# initial var setup
up_down = 0
swing = 0
diff = 2.0
upswing_list = []
downswing_list = []
all_swing_list = []

debug_print = 0

print(datetime.now())
for i, row in df_mini.iterrows():
    if df_mini.index.get_loc(i) == 0:          # initial run through - set entries for first bar
        prev_swing_h = row[1]
        prev_swing_h_loc = i
        prev_swing_h_barnum = 0
        swing_h = row[1]
        swing_hl = row[2]
        swing_h_loc = i
        swing_h_barnum = 0
        swing_h_trend = 1
        prev_swing_l = row[2]
        prev_swing_l_loc = i
        prev_swing_l_barnum = 0 
        swing_l = row[2]
        swing_lh = row[1]
        swing_l_loc = i
        swing_l_barnum = 0
        swing_l_trend = 1
        if debug_print == 1:
            print(f'Row: {df_mini.index.get_loc(i)} Prv Swing High {prev_swing_h} PSHLoc {prev_swing_h_loc}')
            print(f'Row: {df_mini.index.get_loc(i)} Prv Swing Lo {prev_swing_l} PSHLoc {prev_swing_l_loc}')
    ##--------- UP SWING --------------------------------------------------------------------------------#    
    if swing == 1:                 # assume initial upswing - check 
        if row[1] > swing_h:       # if high > crrent hi, calc upswing stats,  make it >, not >= so the first peak is swing hi
            swing_h = row[1]
            swing_hl = row[2]
            swing_h_loc = i
            swing_h_barnum = df_mini.index.get_loc(i)
            up_pts = swing_h - prev_swing_l
            #print(f'Inside Up swing = 1 Row: {df_mini.index.get_loc(i)} Swing High {swing_h} PSHLoc {swing_h_loc} Up Pts: {up_pts}')
            #print(f'Inside Up swing = 1: {df_mini.index.get_loc(i)} Prv Swing Lo {prev_swing_l} PSHLoc {prev_swing_l_loc}')
                            
        if row[10] > 0:   # In Up swing and Down swing hit (lowest x lows crossed) 
            up_pts = swing_h - prev_swing_l
            up_swing_len = swing_h_barnum - prev_swing_l_barnum
            if debug_print == 1:
                print(f'$$ Calc UpSwing Time: Curr Swing Hi B#: {swing_h_barnum} Prev Swing L B#: {prev_swing_l_barnum}')            
                print(f'$$ Calc UpSwing Time: {swing_h_barnum - prev_swing_l_barnum}')
                print("")
                print(f' Low x - Up Swing ends - Row: {df_mini.index.get_loc(i)} Swing High {swing_h} PSHLoc {swing_h_loc} Up Pts: {up_pts}')
                print(f' Low x - Up Swing ends - Row: {df_mini.index.get_loc(i)} Prv Swing Lo {prev_swing_l} PSLLoc {prev_swing_l_loc}')
                print("")
            prev_swing_h = swing_h               # hold prev swing high after new formed down swing
            prev_swing_hl = swing_hl
            prev_swing_h_loc = swing_h_loc       #
            prev_swing_h_barnum = swing_h_barnum #  
            #up_swing_len = 99
            swing = -1                           # now down swing
            swing_l = row[2]                     # reset low to current low
            swing_l_loc = i
            swing_l_barnum = df_mini.index.get_loc(i)
            swing_l_trend = row[6]
            if debug_print == 1:
                print(f' {i} New Down Swing Trail_lox: {row[10]} New Swing Lo: {swing_l}')
                print(f' {i} End of Prev Up Swing: {swing_h} idx: {swing_h_loc}')                
                print('')
            #upswing_list.append(str(swing_h_loc))     # holding swing high - end of up swing
            upswing_list.append({'x0': str(swing_h_loc), 'high':prev_swing_h, 'low':prev_swing_hl})
            if swing_h_trend == 1:
                swing_type = 'LWTr' 
            else:
                swing_type = 'LATr'
            all_swing_list.append({'Date_Time':swing_h_loc, 'trend':swing_h_trend, 'swing':1, 'points':up_pts,'swing_len':up_swing_len,'swing_type':swing_type})
    ##--------- DOWN SWING ------------------------------------------------------------------------------#    
    if swing == -1:                 # if current down swing 
        if row[2] < swing_l:        # new swing low - update current swing
            swing_l = row[2]
            swing_lh = row[1]
            swing_l_loc = i
            swing_l_barnum = df_mini.index.get_loc(i)
            dn_pts = prev_swing_h - swing_l
            if debug_print == 1:
                print(f'Inside Dn swing = 1 Row: {df_mini.index.get_loc(i)} Swing Lo {swing_l} PSLLoc {swing_l_loc} Dn Pts: {dn_pts}')
                print(f'Inside Dn swing = 1: {df_mini.index.get_loc(i)} Prv Swing Hi {prev_swing_h} PSHLoc {prev_swing_h_loc}')
            
            
        if row[8] > 0:   # In Down swing and Up swing hit (highest x highs crossed)
            dn_pts = prev_swing_h - swing_l
            dn_swing_len = swing_l_barnum - prev_swing_h_barnum
            if debug_print == 1:
                print(f'$$ Calc Down Swing Time: Curr Swing Lo B#: {swing_l_barnum} Prev Swing Hi B#: {prev_swing_h_barnum}')            
                print(f'$$ Calc Down Swing Time: {swing_l_barnum - prev_swing_h_barnum}')    
                print("")
                print(f'Hi x - Dn ends - Row: {df_mini.index.get_loc(i)} Swing Lo {swing_l} PSLLoc {swing_l_loc} Dn Pts: {dn_pts}')
                print(f'Hi x - Dn ends - Row: {df_mini.index.get_loc(i)} Prv Swing Hi {prev_swing_h} PSHLoc {prev_swing_h_loc}')
            
            prev_swing_l = swing_l               # hold prev swing low after new formed up swing
            prev_swing_lh = swing_lh
            prev_swing_l_loc = swing_l_loc       #
            prev_swing_l_barnum = swing_l_barnum
            swing = 1                            # now down swing
            swing_h = row[1]                     # reset high to current high
            swing_h_loc = i
            swing_h_barnum = df_mini.index.get_loc(i)
            swing_h_trend = row[6]
            if debug_print == 1:
                print(f' {i}     New Up Swing Trail_hix: {row[8]}')
                print(f' {i}     End of Prev Down Swing: {swing_l} idx: {swing_l_loc}') 
                print('')
            #downswing_list.append(str(swing_l_loc))     # holding swing low - end of down swing
            downswing_list.append({'x0': str(swing_l_loc), 'high':prev_swing_lh, 'low':prev_swing_l})
            if swing_l_trend == -1:
                swing_type = 'SWTr' 
            else:
                swing_type = 'SATr'
            all_swing_list.append({'Date_Time':swing_l_loc, 'trend':swing_l_trend,'swing':-1, 'points':dn_pts, 'swing_len':dn_swing_len,'swing_type':swing_type})

    if df_mini.index.get_loc(i) == 0:     # set up after skipping over initial swing logic
        swing =1
        print(f'Set Swing: {swing}')
    

2019-05-14 16:55:29.750076
Set Swing: 1


In [18]:
#
# print df to text file for testing
#
print(type(all_swing_list))
print(len(all_swing_list))

with open("all_swing_list.txt", "w") as output:
    output.write(str(datetime.now())+"\n")
    for i in range(0,len(all_swing_list)):
        #print('in print loop')
        output.write(str(all_swing_list[i])+"\n")
    

<class 'list'>
286


In [19]:
print(len(upswing_list))
#print(upswing_list)

143


In [20]:
print(len(downswing_list))
#print(downswing_list)

143


In [21]:
# Converting DF index to string for plotly

df_mini_str = []
for i, row in df_mini.iterrows():
    df_mini_str.append(str(i))
    #print(test)

#print(df_mini_str)

In [22]:
# 
# Convert index to string for plotly. PLotly does not plot the time of a bar correctly using the date time stamp index but it does 
# plot correctly with the data time stamp converted to a string
#
df_trend_change_up_plot_str = []
df_trend_change_dn_plot_str = []
 
for i, row in df_trend_change_up_plot.iterrows():
    df_trend_change_up_plot_str.append(str(i))

for i, row in df_trend_change_dn_plot.iterrows():
    df_trend_change_dn_plot_str.append(str(i))

# print(df_trend_change_up_plot_str)
# print("")
#print(df_trend_change_dn_plot_str)

In [23]:
#
# creates line plot list for swing start - plotly needs a dictionary built for each plot
#
blue_dict = {'line':{'color': 'blue', 'width': 1}}
red_dict = {'line':{'color': 'red', 'width': 1}}

line_plot_list = []
zlist = []

def list_line_plot(list_name,color_plot):

    ctr = 0
    z1 = []
    z2 = []
    z3 = []
    z4 = []
    z5 = []
    z6 = []
    z7 = []
    z8 = []
    z9 = []
    z10 = []
    z11 = []

    #zlist = []
    #print(f'inside func - list_name: {list_name} len: {len(list_name)}')
#for item in item_list:
    for item in list_name:
        z1.append('type')
        z2.append('line')
        z3.append('x0')
        z4.append(item["x0"])      # bar date
        z5.append('y0')
        if item["low"] > 0:
            z6.append(item["low"] - 8)
            z10.append(item["low"] - 1)
        else:
            z6.append(2465.00)  
            z10.append(2470.00)  
        z7.append('x1')
        z8.append(item["x0"])      # bar date
        z9.append('y1')
        z11.append('line')

    for i1, i2, i3, i4, i5, i6, i7, i8, i9, i10 in zip(z1, z2, z3, z4, z5, z6, z7, z8, z9, z10):
        zlist.append({i1: i2, i3:i4, i5: i6, i7:i8, i9: i10})

    # print(f'zlist zip len: {len(zlist)} zlist: {zlist} ')    
    for item in zlist:
        item.update(color_plot)
    
    return zlist   # zlist is copied to line_list_plot
    #print(f'zlist zip len: {len(zlist)} zlist: {zlist} ')    
#---------------- End of List_Line_PLot function    

# creete downswing plots
list_line_plot(downswing_list,blue_dict)

# append down swing plots to the line_plot_list
line_plot_list = line_plot_list + zlist

# append upswing swing plots to the line_plot_list
zlist = []
list_line_plot(upswing_list,red_dict)
line_plot_list = line_plot_list + zlist

print_list = 0
if print_list == 1:
    with open("line_plot_list.txt", "w") as output:
        output.write(str(datetime.now())+"\n")
        for i in range(0,len(line_plot_list)):
            #print('in print loop')
            output.write(str(line_plot_list[i])+"\n")

This plotly charts the price data, the trend changes (green dot is a new up trend and a red dot is a new down trend), and the price swings.

The start of an up price swing is shown by a blue line and the start of a down swing is shown by a red line

The chart plots to html as this data is easier to view on a larger plot

In [24]:
# 
# PLotly with multiple plots 
#

trace1 = go.Ohlc(x=df_mini_str, 
                open=df_mini['open'],
                high=df_mini['high'],
                low=df_mini['low'],
                close=df_mini['close'],
                increasing=dict(line=dict(color= '#33CC33')),
                decreasing=dict(line=dict(color= '#CC3300')))

trace2 = go.Scatter(x=df_mini_str,
                            y=df_mini['21ema'],
                            mode='lines',name='21ema')
trace3 = go.Scatter(x=df_mini_str,
                            y=df_mini['55ema'],
                            mode='lines',name='55ema')

trace4 = go.Scatter(x=df_trend_change_up_plot_str,
                            y=df_trend_change_up_plot['Price'],
                            mode='markers',
                            marker = dict(
                            symbol = 'diamond',
                            #    symbol = 'triangle-up',
                            color = '#00ff00',
                            size = 10,
                            line = dict(width = 1)),
                            name='Up Trend')

trace5 = go.Scatter(x=df_trend_change_dn_plot_str,
                            y=df_trend_change_dn_plot['Price'],
                            mode='markers',
                            marker = dict(
                            symbol = 'diamond',
                            #    symbol = 'triangle-up',
                            color = '#ff0000',
                            size = 10,
                            line = dict(width = 1)),
                            name='Dn Trend')

layout = go.Layout(autosize=True,
                xaxis=go.layout.XAxis(rangeslider=dict(visible=False)),
                yaxis=go.layout.YAxis(automargin=True,autorange=True),
                shapes=line_plot_list)                 # zlist is updated below 
                  #shapes=zlist)                 # zlist is updated below

#print(trace1)
data = [trace1,trace2,trace3,trace4,trace5]

fig = go.Figure(data=data,layout=layout)
pyo.plot(fig, filename='styled_ohlc.html')

'file://C:\\Users\\522147\\Documents\\1 Python\\1Yves - Python for Finance\\Latest Files from TPQ\\styled_ohlc.html'

# TEST CODE BELOW

In [25]:
# pring off swings

for i in range(0,5):
    print(all_swing_list[i])

{'Date_Time': Timestamp('2019-01-18 00:20:20'), 'trend': 1, 'swing': 1, 'points': 0.0, 'swing_len': 0, 'swing_type': 'LWTr'}
{'Date_Time': Timestamp('2019-01-18 00:20:40'), 'trend': 1.0, 'swing': -1, 'points': 0.5, 'swing_len': 1, 'swing_type': 'SATr'}
{'Date_Time': Timestamp('2019-01-18 00:21:20'), 'trend': 1.0, 'swing': 1, 'points': 0.75, 'swing_len': 2, 'swing_type': 'LWTr'}
{'Date_Time': Timestamp('2019-01-18 00:33:20'), 'trend': 1.0, 'swing': -1, 'points': 1.25, 'swing_len': 36, 'swing_type': 'SATr'}
{'Date_Time': Timestamp('2019-01-18 00:35:40'), 'trend': 1.0, 'swing': 1, 'points': 0.5, 'swing_len': 7, 'swing_type': 'LWTr'}


In [26]:
test_df = pd.DataFrame(all_swing_list)

In [27]:
print(test_df.head(10))

            Date_Time  points  swing  swing_len swing_type  trend
0 2019-01-18 00:20:20    0.00      1          0       LWTr    1.0
1 2019-01-18 00:20:40    0.50     -1          1       SATr    1.0
2 2019-01-18 00:21:20    0.75      1          2       LWTr    1.0
3 2019-01-18 00:33:20    1.25     -1         36       SATr    1.0
4 2019-01-18 00:35:40    0.50      1          7       LWTr    1.0
5 2019-01-18 00:38:40    0.75     -1          9       SATr    1.0
6 2019-01-18 00:39:20    0.75      1          2       LWTr    1.0
7 2019-01-18 00:41:00    0.75     -1          5       SATr    1.0
8 2019-01-18 00:51:00    1.50      1         30       LWTr    1.0
9 2019-01-18 00:52:40    0.50     -1          5       SATr    1.0


In [28]:
test_df[['points','swing_len']].describe(percentiles=[.1, .2, .3, .4, .5, .75, .80, .90])

,points,swing_len
count,286.000000,286.000000
mean,2.333916,10.430070
std,1.627058,7.876258
min,0.000000,0.000000
10%,0.750000,3.000000
20%,1.000000,4.000000
30%,1.250000,5.000000
40%,1.500000,7.000000
50%,2.000000,8.000000
75%,3.000000,14.000000


In [29]:
df_LWTr = test_df[test_df['swing_type'] == 'LWTr']
df_LWTr.head(10)

,Date_Time,points,swing,swing_len,swing_type,trend
0,2019-01-18 00:20:20,0.00,1,0,LWTr,1.0
2,2019-01-18 00:21:20,0.75,1,2,LWTr,1.0
4,2019-01-18 00:35:40,0.50,1,7,LWTr,1.0
6,2019-01-18 00:39:20,0.75,1,2,LWTr,1.0
8,2019-01-18 00:51:00,1.50,1,30,LWTr,1.0
10,2019-01-18 00:54:40,0.50,1,6,LWTr,1.0
12,2019-01-18 00:59:20,0.75,1,10,LWTr,1.0
14,2019-01-18 01:00:20,1.25,1,1,LWTr,1.0
16,2019-01-18 01:04:40,0.75,1,6,LWTr,1.0
18,2019-01-18 01:10:20,0.75,1,5,LWTr,1.0


In [30]:
df_LATr = test_df[test_df['swing_type'] == 'LATr']
df_LATr.head(10)

,Date_Time,points,swing,swing_len,swing_type,trend
36,2019-01-18 02:33:20,1.50,1,39,LATr,-1.0
38,2019-01-18 02:38:40,1.25,1,5,LATr,-1.0
40,2019-01-18 02:47:00,2.75,1,14,LATr,-1.0
48,2019-01-18 03:17:00,3.75,1,12,LATr,-1.0
58,2019-01-18 03:43:00,2.00,1,2,LATr,-1.0
60,2019-01-18 03:52:00,2.50,1,19,LATr,-1.0
62,2019-01-18 03:54:40,1.25,1,3,LATr,-1.0
64,2019-01-18 03:59:00,1.75,1,9,LATr,-1.0
66,2019-01-18 04:04:40,0.75,1,6,LATr,-1.0
68,2019-01-18 04:09:40,2.25,1,11,LATr,-1.0


In [33]:
df_SWTr = test_df[test_df['swing_type'] == 'SWTr']
df_SWTr.head(10)

,Date_Time,points,swing,swing_len,swing_type,trend
35,2019-01-18 02:20:20,1.50,-1,2,SWTr,-1.0
37,2019-01-18 02:37:00,2.50,-1,11,SWTr,-1.0
39,2019-01-18 02:42:20,0.75,-1,11,SWTr,-1.0
59,2019-01-18 03:45:40,2.00,-1,8,SWTr,-1.0
61,2019-01-18 03:53:40,1.25,-1,5,SWTr,-1.0
63,2019-01-18 03:56:00,1.25,-1,4,SWTr,-1.0
65,2019-01-18 04:02:40,1.75,-1,11,SWTr,-1.0
67,2019-01-18 04:06:00,1.00,-1,4,SWTr,-1.0
69,2019-01-18 04:13:40,1.50,-1,12,SWTr,-1.0
89,2019-01-18 05:36:40,2.00,-1,6,SWTr,-1.0


In [34]:
df_SATr = test_df[test_df['swing_type'] == 'SATr']
df_SATr.head(10)

,Date_Time,points,swing,swing_len,swing_type,trend
1,2019-01-18 00:20:40,0.50,-1,1,SATr,1.0
3,2019-01-18 00:33:20,1.25,-1,36,SATr,1.0
5,2019-01-18 00:38:40,0.75,-1,9,SATr,1.0
7,2019-01-18 00:41:00,0.75,-1,5,SATr,1.0
9,2019-01-18 00:52:40,0.50,-1,5,SATr,1.0
11,2019-01-18 00:56:00,0.50,-1,4,SATr,1.0
13,2019-01-18 01:00:00,0.50,-1,2,SATr,1.0
15,2019-01-18 01:02:40,1.25,-1,7,SATr,1.0
17,2019-01-18 01:08:40,0.75,-1,12,SATr,1.0
19,2019-01-18 01:15:40,0.75,-1,16,SATr,1.0


In [31]:
#
# Plotting diff swing tyoes
# 
trace0 = go.Scatter(
    y = df_LWTr['points'],
    x = df_LWTr['swing_len'],
    name = 'LWTr',
    mode = 'markers',
    marker = dict(
        size = 10,
        color = 'rgba(152, 0, 0, .8)',
        line = dict(
            width = 2,
            color = 'rgb(0, 0, 0)'
        )
    )
)

trace1 = go.Scatter(
    y = df_LATr['points'],
    x = df_LATr['swing_len'],
    name = 'LATr',
    mode = 'markers',
    marker = dict(
        size = 10,
        color = 'rgba(255, 182, 193, .9)',
        line = dict(
            width = 2,
        )
    )
)

layout = go.Layout(autosize=True,
                xaxis=go.layout.XAxis(rangeslider=dict(visible=False)),
                yaxis=go.layout.YAxis(automargin=True,autorange=True))
#                shapes=line_plot_list)                 # zlist is updated below 
                  #shapes=zlist)                 # zlist is updated below

#print(trace1)
data = [trace0,trace1]

fig = go.Figure(data=data,layout=layout)
pyo.plot(fig, filename='With and Against Trend.html')

'file://C:\\Users\\522147\\Documents\\1 Python\\1Yves - Python for Finance\\Latest Files from TPQ\\With and Against Trend.html'

In [36]:
#
# Plotting diff swing types - Short Trades
# 
trace0 = go.Scatter(
    y = df_SWTr['points'],
    x = df_SWTr['swing_len'],
    name = 'SWTr',
    mode = 'markers',
    marker = dict(
        size = 10,
        color = 'rgba(152, 0, 0, .8)',
        line = dict(
            width = 2,
            color = 'rgb(0, 0, 0)'
        )
    )
)

trace1 = go.Scatter(
    y = df_SATr['points'],
    x = df_SATr['swing_len'],
    name = 'SATr',
    mode = 'markers',
    marker = dict(
        size = 10,
        color = 'rgba(255, 182, 193, .9)',
        line = dict(
            width = 2,
        )
    )
)

layout = go.Layout(autosize=True,
                xaxis=go.layout.XAxis(rangeslider=dict(visible=False)),
                yaxis=go.layout.YAxis(automargin=True,autorange=True))
#                shapes=line_plot_list)                 # zlist is updated below 
                  #shapes=zlist)                 # zlist is updated below

#print(trace1)
data = [trace0,trace1]

fig = go.Figure(data=data,layout=layout)
pyo.plot(fig, filename='Short Trades - With and Against Trend.html')

'file://C:\\Users\\522147\\Documents\\1 Python\\1Yves - Python for Finance\\Latest Files from TPQ\\Short Trades - With and Against Trend.html'

In [48]:
import plotly.figure_factory as ff

import numpy as np

# Add histogram data
x1 = df_LWTr['points'] 
x2 = df_LATr['points']

# Group data together
hist_data = [x1, x2]

group_labels = ['LWTr', 'LATr']

# Create distplot with custom bin_size
fig = ff.create_distplot(hist_data, group_labels, bin_size=[.25, .5, .75, 1, 1.25, 1.5, 1.75, 2.0, 2.5, 3.0, 4.0])

# Plot!
pyo.plot(fig, filename='Long Trades - Distplot with Multiple Bin Sizes.html')

'file://C:\\Users\\522147\\Documents\\1 Python\\1 - Yves - Python for Finance\\Latest Files from TPQ\\Distplot with Multiple Bin Sizes.html'

In [35]:
import plotly.figure_factory as ff

import numpy as np

# Add histogram data
x3 = df_SWTr['points'] 
x4 = df_SATr['points']

# Group data together
hist_data = [x3, x4]

group_labels = ['SWTr', 'SATr']

# Create distplot with custom bin_size
fig = ff.create_distplot(hist_data, group_labels, bin_size=[.25, .5, .75, 1, 1.25, 1.5, 1.75, 2.0, 2.5, 3.0, 4.0])

# Plot!
pyo.plot(fig, filename='Short Trades - Distplot with Multiple Bin Sizes.html')

'file://C:\\Users\\522147\\Documents\\1 Python\\1Yves - Python for Finance\\Latest Files from TPQ\\Short Trades - Distplot with Multiple Bin Sizes.html'

In [28]:
df_LWTr_filt = test_df['swing_type'] == 'LWTr'
#testdf[test_df_uptrend_filt].describe(include=['points','swing_len'],percentiles=[.1, .2, .3, .4, .5, .75, .80, .90])

In [30]:
test_df[df_LWTr_filt]

,Date_Time,points,swing,swing_len,swing_type,trend
0,2019-01-18 00:20:20,0.00,1,0,LWTr,1.0
2,2019-01-18 00:21:20,0.75,1,2,LWTr,1.0
4,2019-01-18 00:35:40,0.50,1,7,LWTr,1.0
6,2019-01-18 00:39:20,0.75,1,2,LWTr,1.0
8,2019-01-18 00:51:00,1.50,1,30,LWTr,1.0
10,2019-01-18 00:54:40,0.50,1,6,LWTr,1.0
12,2019-01-18 00:59:20,0.75,1,10,LWTr,1.0
14,2019-01-18 01:00:20,1.25,1,1,LWTr,1.0
16,2019-01-18 01:04:40,0.75,1,6,LWTr,1.0
18,2019-01-18 01:10:20,0.75,1,5,LWTr,1.0


In [31]:
zdf_LWtr = test_df[df_LWTr_filt]

In [33]:
zdf_LWtr.head()

,Date_Time,points,swing,swing_len,swing_type,trend
0,2019-01-18 00:20:20,0.00,1,0,LWTr,1.0
2,2019-01-18 00:21:20,0.75,1,2,LWTr,1.0
4,2019-01-18 00:35:40,0.50,1,7,LWTr,1.0
6,2019-01-18 00:39:20,0.75,1,2,LWTr,1.0
8,2019-01-18 00:51:00,1.50,1,30,LWTr,1.0


In [38]:
zdf2_LATr = test_df[test_df['swing_type'] == 'SATr']
zdf2_LATr

,Date_Time,points,swing,swing_len,swing_type,trend
1,2019-01-18 00:20:40,0.50,-1,1,SATr,1.0
3,2019-01-18 00:33:20,1.25,-1,36,SATr,1.0
5,2019-01-18 00:38:40,0.75,-1,9,SATr,1.0
7,2019-01-18 00:41:00,0.75,-1,5,SATr,1.0
9,2019-01-18 00:52:40,0.50,-1,5,SATr,1.0
11,2019-01-18 00:56:00,0.50,-1,4,SATr,1.0
13,2019-01-18 01:00:00,0.50,-1,2,SATr,1.0
15,2019-01-18 01:02:40,1.25,-1,7,SATr,1.0
17,2019-01-18 01:08:40,0.75,-1,12,SATr,1.0
19,2019-01-18 01:15:40,0.75,-1,16,SATr,1.0


In [51]:
test_df.loc[(test_df['trend'] == 1 & test_df['swing'] == 1), 'swing_type'] = 'LWTr'

ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

In [ ]:
# trend update - need to update trend in list (based on rules), turn list into DF and merge back with main df
#df.reset_index(drop=True)
df_test = df.iloc[-3000:].copy()

In [ ]:
print(df_test.head())

In [ ]:
# Converting DF index to string for plotly
# 
ztest_str = []
# ytest_str = 
ctr = 0
#for i, row in test2_df.iterrows():
for i, row in test_trend_df.iterrows():
    ctr += 1
    #print(f'Ctr: {ctr} at {i}')
    ztest_str.append(str(i))
    #print(test)

for i in range(0,5):
    print(ztest_str[i])
    
#print(xtest_str[0])
print(len(ztest_str))

In [ ]:
#print(trend_change_up_plot)
#print(trend_change_dn_plot)
#print(trend_change_flag)

In [ ]:
# Convert test_trend_list to DF
testz_df = pd.DataFrame.from_dict(test_trend_list,orient='index',columns=['Trend','Price'])
testz_df.index.names= ['Date_Time']  
print(testz_df)
print(testz_df.index)

In [ ]:
# Converting DF index to string for plotly

xtest_str = []
#ytest_str = 
ctr = 0
for i, row in df_test.iterrows():
    ctr += 1
    #print(f'Ctr: {ctr} at {i}')
    xtest_str.append(str(i))
    #print(test)

for i in range(0,10):
    print(xtest_str[i])
    
#print(xtest_str[0])
print(len(xtest_str))

In [ ]:
df_test.info()

In [ ]:
# 

In [ ]:
# Test DF for plotting buys / sells
test_list = {'2019-01-18 14:00:00' : 2640.5, '2019-01-18 14:05:00' : 2640, '2019-01-18 14:10:00' : 2641.75,'2019-01-18 14:15:00' : 2640}
print(test_list)
test_list['2019-01-18 14:20:00'] = 2640.5
print(test_list)

In [ ]:
test_df = pd.DataFrame(list(test_list.items()), columns=['Date_Time', 'BuyPrice'])
test_df.set_index('Date_Time')  
print(test_df)
print(test_df.index)

In [ ]:
test2_df = pd.DataFrame.from_dict(test_list, orient='index', columns=['BuyPr'])
print(test2_df)
print(test2_df.index)
test2_df.index  = pd.DatetimeIndex(test2_df.index)
print("")
print(test2_df)
print(test2_df.index)

# df.index = pd.DatetimeIndex(df.index)
print(df_test.index)
'''>>> pd.DataFrame.from_dict(data, orient='index',
...                        columns=['A', 'B', 'C', 'D'])
       A  B  C  D
row_1  3  2  1  0
row_2  a  b  c  d
'''

In [ ]:
df_mini.set_index('Date_Time',inplace=True)

In [ ]:
print(df_mini.index)

In [ ]:
test_trend_df = pd.DataFrame.from_dict(test_trend, orient='index', columns=['BuyPr'])
print(test_trend_df)
print(test_trend_df.index)
print(test_trend_df.info())
#test2_df.index  = pd.DatetimeIndex(test2_df.index)
#print("")
#print(test2_df)
#print(test2_df.index)

# df.index = pd.DatetimeIndex(df.index)
#print(df_test.index)
'''>>> pd.DataFrame.from_dict(data, orient='index',
...                        columns=['A', 'B', 'C', 'D'])
       A  B  C  D
row_1  3  2  1  0
row_2  a  b  c  d
'''

In [ ]:
# Converting DF index to string for plotly

ztest_str = []
# ytest_str = 
ctr = 0
#for i, row in test2_df.iterrows():
for i, row in test_trend_df.iterrows():
    ctr += 1
    #print(f'Ctr: {ctr} at {i}')
    ztest_str.append(str(i))
    #print(test)

for i in range(0,5):
    print(ztest_str[i])
    
#print(xtest_str[0])
print(len(ztest_str))

In [ ]:
# Testing Plots
#
# testing plots to plot buy / sell sigs and exits
#
# 
# testing plotly example
#print(df.index)
#x = df.index.to_pydatetime()
#print(x)
trace1 = go.Ohlc(x=df_mini.index,
                open=df_mini['open'],
                high=df_mini['high'],
                low=df_mini['low'],
                close=df_mini['close'],
                increasing=dict(line=dict(color= '#33CC33')),
                decreasing=dict(line=dict(color= '#CC3300')))
#
#trace2 = go.Scatter(x=ztest_str,
#                    y=test2_df['BuyPr'],
#                   mode='markers',name='BuyPr')
'''
trace2 = go.Scatter(x=ztest_str,
                           # y=test2_df['BuyPr'],
                            y=test_trend_df['BuyPr'],
                            mode='markers',
                            marker = dict(
                            symbol = 'triangle-up',
                            color = 'blue',
                            size = 8,
                            line = dict(width = 2)),
                            name='BuyPr')
'''
# marker={'color': 'blue', 'symbol': 'star', 'size': 10}
#trace3 = go.Scatter(x=x_str,
#                            y=df_mini['55ema'],
#                            mode='lines',name='55ema')

layout = go.Layout(autosize=True,
                xaxis=go.layout.XAxis(rangeslider=dict(visible=False)),
                yaxis=go.layout.YAxis(automargin=True,autorange=True))
 #                 shapes=line_plot_list)                 # zlist is updated below 
                  #shapes=zlist)                 # zlist is updated below

#print(trace1)
#data = [trace1,trace2]
data = [trace1]

fig = go.Figure(data=data,layout=layout)
pyo.plot(fig, filename='test')
''' 
import plotly.plotly as py
import plotly.graph_objs as go
import numpy as np

N = 100000
trace = go.Scattergl(
    x = np.random.randn(N),
    y = np.random.randn(N),
    mode = 'markers',
    marker = dict(
        color = '#FFBAD2',
        line = dict(width = 1)
    )
)
data = [trace]
py.iplot(data, filename='compare_webgl')
'''

In [ ]:
trace1 = go.Scatter(
    x = np.random.randn(N),
    y = np.random.randn(N)-2,
    name = 'Below',
    mode = 'markers',
    marker = dict(
        size = 10,
        color = 'rgba(255, 182, 193, .9)',
        line = dict(
            width = 2,
        )
    )
)

In [ ]:
df_test2 = df.iloc[-3000:].copy()
df_test2.head(50)

In [ ]:
print(test_trend_df)

In [ ]:
#print(df_test2.head())
df_test2.head(20)

In [ ]:

test_trend_df.index.rename('Date_Time',inplace=True)
print(test_trend_df.head())
#df.index.rename('j',inplace=True)
#print(test_trend_df.index)
test_trend_df.head()

In [ ]:
print(df_test2.index)

In [ ]:
# This works to merge
#
df_merged = pd.merge_ordered(df_test2, test_trend_df, on='Date_Time',how='left',fill_method='ffill')
#pd.set_option("display.max_columns", 30)
df_merged

In [ ]:
# Test merge with specific columns
# This works to merge
#
df_merged2 = pd.merge_ordered(df_mini, df_trend_change_flag, on='Date_Time',how='left',fill_method='ffill')
pd.set_option("display.max_rows", 400)
df_merged2.head(20)

In [ ]:
#df_merged2.drop(columns=['Price'],inplace=True)
df_merged2.tail(400)
#gapminder_ocean.drop(['pop'], axis=1)

In [ ]:
# testing merge (to update)
#
# df_merge2 - has trend and price
# want to know if I can just plot the high or low price based on shift(1) trend != current trend
#ema_delta = 1.5

df_merged2['test_trend'] = np.where(df_merged2['Trend'] != df_merged2['Trend'].shift(1), df_merged2['Price'],'')
#
#df_mini['trail_hi'] = df_mini['high'].rolling(5,min_periods=1).max()
#df_mini['trail_hix'] = df_mini['high'] - df_mini['trail_hi'].shift(1)
#df_mini['trail_lo'] = df_mini['low'].rolling(5,min_periods=1).min()
#df_mini['trail_lox'] = df_mini['trail_lo'].shift(1) - df_mini['low']        #rolling includes current bar - need prev row value
#df_mini.tail(1000)
#for i in range(200,400):
#    print(df_mini.iloc[i])

In [ ]:
df_merged2.head(200)

In [ ]:
# Testing Plots
#
# testing plots to plot buy / sell sigs and exits
#
# 
# testing plotly example
#print(df.index)
#x = df.index.to_pydatetime()
#print(x)
trace1 = go.Ohlc(x=xtest_str,
                open=df_merged2['open'],
                high=df_merged2['high'],
                low=df_merged2['low'],
                close=df_merged2['close'],
                increasing=dict(line=dict(color= '#33CC33')),
                decreasing=dict(line=dict(color= '#CC3300')))
#
#trace2 = go.Scatter(x=ztest_str,
#                    y=test2_df['BuyPr'],
#                   mode='markers',name='BuyPr')

trace2 = go.Scatter(x=xtest_str,
                           # y=test2_df['BuyPr'],
                            y=df_merged2['Price'],
                            mode='markers',
                            marker = dict(
                            symbol = 'triangle-up',
                            color = 'green',
                            size = 8,
                            line = dict(width = 2)),
                            name='Trend Change')
# marker={'color': 'blue', 'symbol': 'star', 'size': 10}
#trace3 = go.Scatter(x=x_str,
#                            y=df_mini['55ema'],
#                            mode='lines',name='55ema')

layout = go.Layout(autosize=True,
                xaxis=go.layout.XAxis(rangeslider=dict(visible=False)),
                yaxis=go.layout.YAxis(automargin=True,autorange=True))
 #                 shapes=line_plot_list)                 # zlist is updated below 
                  #shapes=zlist)                 # zlist is updated below

#print(trace1)
data = [trace1,trace2]
#data = [trace1]

fig = go.Figure(data=data,layout=layout)
pyo.plot(fig, filename='test')
''' 
import plotly.plotly as py
import plotly.graph_objs as go
import numpy as np

N = 100000
trace = go.Scattergl(
    x = np.random.randn(N),
    y = np.random.randn(N),
    mode = 'markers',
    marker = dict(
        color = '#FFBAD2',
        line = dict(width = 1)
    )
)
data = [trace]
py.iplot(data, filename='compare_webgl')
'''